In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection  import cross_val_score,RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv',index_col='PassengerId')
test=pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv',index_col='PassengerId')

submit=pd.DataFrame(test.index)


In [ ]:
train=pd.get_dummies(train,columns=['Sex','Embarked'],drop_first=True)
train.drop(['Name','Ticket','Cabin'],inplace=True,axis=1)


#---------------------------------------------------------------------------------------------------

test=pd.get_dummies(test,columns=['Sex','Embarked'],drop_first=True)
test.drop(['Name','Ticket','Cabin'],inplace=True,axis=1)

In [ ]:
pipeline=Pipeline(steps=[('i',KNNImputer(n_neighbors=9)),('rfe',RFECV(estimator=GradientBoostingClassifier())),('model',GradientBoostingClassifier())])
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3)
score=cross_val_score(pipeline,train.drop('Survived',axis=1),train['Survived'],cv=cv,scoring='accuracy',n_jobs=-1)
np.mean(score)

In [ ]:
pipeline.fit(train.drop('Survived',axis=1),train['Survived'])
            
submit['Survived']=pipeline.predict(test)
submit.to_csv('ver1.csv',index=False)
            
        
    

In [ ]:
submit